In [ ]:
#Importing libraries
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from urllib.error import ContentTooShortError
import requests
import re
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import time
import sys
from urllib3 import HTTPConnectionPool
import base64
import urllib
import socket
import urllib3
from urllib3.exceptions import ProtocolError
from urllib3.exceptions import ReadTimeoutError
from urllib3.exceptions import DecodeError
from requests.exceptions import MissingSchema, InvalidURL, ChunkedEncodingError,ContentDecodingError, ConnectionError, StreamConsumedError
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.common.exceptions import TimeoutException

In [ ]:
#options = Options()
#options.add_argument("--headless")
#driver = webdriver.Chrome(options=options)
driver = webdriver.PhantomJS()
#driver = webdriver.Firefox()

In [ ]:
#options = FirefoxOptions()
#options.add_argument("--headless")
#driver = webdriver.Firefox(options=options)

In [ ]:
dataset = pd.read_csv('top-1m.csv')
dataset.shape

In [ ]:
#dataset_f = dataset[22:222]
dataset_f = dataset[:500]

In [ ]:
#Save to file
dataset_f.to_csv("apriori.csv",index=False)

In [ ]:
dataset_f = pd.read_csv('apriori.csv')
dataset_f.shape

In [ ]:
dataset_f.head(10)

In [ ]:
#1. Line Counts
def line_count(url):
    if not re.match(r"^https?", url):
         url = "http://" + url
    try:
        r = requests.get(url, stream = True)
        count=0
        for i in r.iter_lines():
            if i:
                count=count+1
        return count
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return -1    
    except requests.ConnectionError as e:
        return -1
    except UnicodeError as e:
        return -1  
    except ConnectionResetError as e:
        return -1
    except OSError as e:
        return -1
    except ProtocolError as e:
        return -1
    except ReadTimeoutError  as e:
        return -1
    except RuntimeError as e:
        return -1
    except ChunkedEncodingError as e:
        return -1
    except ReadTimeoutError as e:
        return -1
    except DecodeError as e:
        return -1
    except ContentDecodingError as e:
        return -1

In [ ]:
#2. Max Length
def max_length_line_count(url):
    if not re.match(r"^https?", url):
         url = "http://" + url
    try:
        r = requests.get(url, stream = True)
        max_length = 0
        max_len_line= ''
        for i in r.iter_lines():
            if (len(i)>max_length):
                max_length=len(i)
                max_len_line = i
        return(len(max_len_line))
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return -1    
    except requests.ConnectionError as e:
        return -1
    except UnicodeError as e:
        return -1
    except ConnectionResetError as e:
        return -1
    except OSError as e:
        return -1
    except ProtocolError as e:
        return -1
    except ReadTimeoutError  as e:
        return -1
    except RuntimeError as e:
        return -1
    except ChunkedEncodingError as e:
        return -1
    except ReadTimeoutError as e:
        return -1
    except DecodeError as e:
        return -1
    except ContentDecodingError as e:
        return -1

In [ ]:
#Requesting to get access to webpage
def response_check(url):
    if not re.match(r"^https?", url):
         url = "http://" + url
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        return response
        #webpage = urlopen(req).read()
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return False    
    except requests.ConnectionError:
        return False
    except UnicodeError:
        return False
    except ConnectionResetError as e:
        return False
    except OSError as e:
        return False
    except ProtocolError as e:
        return False
    except ReadTimeoutError  as e:
        return False
    except RuntimeError as e:
        return False
    except ChunkedEncodingError as e:
        return False
    except ReadTimeoutError as e:
        return False
    except DecodeError as e:
        return False
    except ContentDecodingError as e:
        return False
    except AttributeError as e:
        return False

In [ ]:
#3. Checking for abnormal value for elements
"""Attackers use invisible attributes to hide their elements and set the width
and height of elements used to deliver the attack to small values"""
def elements_small_area(response):
    if(response!=False):
        res = BeautifulSoup(response.text, 'html.parser')
        count = 0
        types = ['div','iframe','object']
        for t in types:
            elements = res.find_all(t, height = True, width = True)
            for e in elements:
                attrs = e.attrs
                try:
                    h = int(attrs['height'])
                    w = int(attrs['width'])
                    if h <= 2 or w <= 2:
                        count += 1
                        continue
                    if h * w <= 30:
                        count += 1
                except:
                    continue
        return count 
    else:
        return -1

In [ ]:
#4. WhiteSpace Ratio
def whitespace_count(response):
    if(response!=False):
        char_count = len(response.text)
        r = BeautifulSoup(response.text, 'html.parser')
        try:
            white_space = str(r).count('')
            return float(white_space)/float(char_count)
        except ZeroDivisionError as e:
            return 0  
    else:
        return -1

In [ ]:
#5. External JavaScripts
def external_javascript_file_count(url):
    if not re.match(r"^https?", url):
        url = "http://" + url
    try:
        driver.get(url)
        page = BeautifulSoup(driver.page_source,"html.parser")
        return len(page.findAll("script",src= True))
    except TimeoutException as e:
        return -1 
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return -1    
    except requests.ConnectionError as e:
        return -1
    except UnicodeError as e:
        return -1  
    except ConnectionResetError as e:
        return -1
    except OSError as e:
        return -1
    except ProtocolError as e:
        return -1
    except ReadTimeoutError  as e:
        return -1
    except RuntimeError as e:
        return -1
    except ChunkedEncodingError as e:
        return -1
    except ReadTimeoutError as e:
        return -1
    except DecodeError as e:
        return -1
    except ContentDecodingError as e:
        return -1      

    
    
        
    

In [ ]:
#6. num JavaScripts
def javascript_count(url):
    if not re.match(r"^https?", url):
        url = "http://" + url
    try:
        driver.get(url)
        page = BeautifulSoup(driver.page_source,"html.parser")
        return len(page.findAll("script"))
    except TimeoutException as e:
        return -1 
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return -1    
    except requests.ConnectionError as e:
        return -1
    except UnicodeError as e:
        return -1  
    except ConnectionResetError as e:
        return -1
    except OSError as e:
        return -1
    except ProtocolError as e:
        return -1
    except ReadTimeoutError  as e:
        return -1
    except RuntimeError as e:
        return -1
    except ChunkedEncodingError as e:
        return -1
    except ReadTimeoutError as e:
        return -1
    except DecodeError as e:
        return -1
    except ContentDecodingError as e:
        return -1        


In [ ]:
#7. Number of iframes
def iframe_count(url):
    if not re.match(r"^https?", url):
        url = "http://" + url
    try:
        driver.get(url)
        page = BeautifulSoup(driver.page_source,"html.parser")
        return len(page.findAll("iframe"))
    except TimeoutException as e:
        return -1
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return -1    
    except requests.ConnectionError as e:
        return -1
    except UnicodeError as e:
        return -1  
    except ConnectionResetError as e:
        return -1
    except OSError as e:
        return -1
    except ProtocolError as e:
        return -1
    except ReadTimeoutError  as e:
        return -1
    except RuntimeError as e:
        return -1
    except ChunkedEncodingError as e:
        return -1
    except ReadTimeoutError as e:
        return -1
    except DecodeError as e:
        return -1
    except ContentDecodingError as e:
        return -1         
  

In [ ]:
#8. Obfuscation and Suspicious functions
def extract_obfus_func(url):
    if not re.match(r"^https?", url):
        url = "http://" + url
    try:
        driver.get(url)
        Obfus_functions = ['eval(','fromCharCode(', 'unescape(','escape(','Math.random',
                      'ADODB.Stream','CreateObject(','WScript.Shell','FileSystemOject(','FileExists(',
                       'charAt(','charCodeAt(','substr(']
        count = 0
        for t in Obfus_functions:
            elements = driver.page_source.count(t)
            count += elements
        return count      
    except TimeoutException as e:
        return -1 
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return -1    
    except requests.ConnectionError as e:
        return -1
    except UnicodeError as e:
        return -1  
    except ConnectionResetError as e:
        return -1
    except OSError as e:
        return -1
    except ProtocolError as e:
        return -1
    except ReadTimeoutError  as e:
        return -1
    except RuntimeError as e:
        return -1
    except ChunkedEncodingError as e:
        return -1
    except ReadTimeoutError as e:
        return -1
    except DecodeError as e:
        return -1
    except ContentDecodingError as e:
        return -1        
    
  
   
    
        
       
           
            
            
        
         
            
            

In [ ]:
#9. frequency of var and function
def freq_var_function(url):
    if not re.match(r"^https?", url):
        url = "http://" + url
    try:
        driver.get(url)
        keyword_func = ['var','function']
        count = 0
        for t in keyword_func:
            func = driver.page_source.count(t)
            count += func
        return count      
    except TimeoutException as e:
        return -1
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return -1    
    except requests.ConnectionError as e:
        return -1
    except UnicodeError as e:
        return -1  
    except ConnectionResetError as e:
        return -1
    except OSError as e:
        return -1
    except ProtocolError as e:
        return -1
    except ReadTimeoutError  as e:
        return -1
    except RuntimeError as e:
        return -1
    except ChunkedEncodingError as e:
        return -1
    except ReadTimeoutError as e:
        return -1
    except DecodeError as e:
        return -1
    except ContentDecodingError as e:
        return -1                  
            
        

In [ ]:
#10. DOM Methods
def dom_methods(url):
    if not re.match(r"^https?", url):
        url = "http://" + url
    try:
        driver.get(url)
        dom_elements = ['appendChild(','insertBefore(','createElement(','getAttribute(','getElementById(',
                   'getElementsByTagName(','parentNode','cloneNode(','setAttribute(','getElementsByClassName(']
        count = 0 
        for t in dom_elements:
            dom =  driver.page_source.count(t)
            count += dom
        return count
    except TimeoutException as e:
        return -1 
    except (URLError,HTTPError,ContentTooShortError)  as e:
        return -1    
    except requests.ConnectionError as e:
        return -1
    except UnicodeError as e:
        return -1  
    except ConnectionResetError as e:
        return -1
    except OSError as e:
        return -1
    except ProtocolError as e:
        return -1
    except ReadTimeoutError  as e:
        return -1
    except RuntimeError as e:
        return -1
    except ChunkedEncodingError as e:
        return -1
    except ReadTimeoutError as e:
        return -1
    except DecodeError as e:
        return -1
    except ContentDecodingError as e:
        return -1               


    
   
   
    
        
        
        

            
         

In [ ]:
#Create an array to store values
line_counts = []
max_length_line = []
small_area= []
whitespace_counts = []
external_javascript_count = []
javascript_counts = []
iframe_counts = []
Obfus_Suspicious_functions = []
var_function = []
dom_method = []


In [ ]:
#Extracting features
def extract():
    counter = 0
    for url in dataset_f['URL']:
        counter = counter + 1
        print(counter)
        response = response_check(url)
        iframe_counts.append(iframe_count(url))
        javascript_counts.append(javascript_count(url))
        small_area.append(elements_small_area(response))
        line_counts.append(line_count(url))
        max_length_line.append(max_length_line_count(url))
        whitespace_counts.append(whitespace_count(response))
        external_javascript_count.append(external_javascript_file_count(url))
        Obfus_Suspicious_functions.append(extract_obfus_func(url))
        var_function.append(freq_var_function(url))
        dom_method.append(dom_methods(url))

In [ ]:
extract()

In [ ]:
#Checking if extraction was done
print(f'iframe              : {len(iframe_counts)}')
print(f'small area              : {len(small_area)}')
print(f'line_counts             : {len(line_counts)}')
print(f'maximum length of line             : {len(max_length_line)}')
print(f'whitespace_counts              : {len(whitespace_counts)}')
print(f'external JavaScript              : {len(external_javascript_count)}')
print(f'Number of JavaScript              : {len(javascript_counts)}')
print(f'The number of Obfuscation functions in this url: {len(Obfus_Suspicious_functions)}')
print(f'The number of var and function() in this url: {len(var_function)}')
print(f'The number of dom_methods() in this url: {len(dom_method)}')

In [ ]:
#Create a dataframe to add extracted features columns previous dataset 
features_df = pd.DataFrame(dataset_f)
features_df['iframe_count'] = iframe_counts
features_df['small_area'] = small_area
features_df['line_counts'] = line_counts
features_df['max_length_line'] = max_length_line
features_df['space_%'] = whitespace_counts
features_df['js_count'] = javascript_counts 
features_df['ext_js_count'] = external_javascript_count
features_df['obfus_suspicious_func_count'] = Obfus_Suspicious_functions 
features_df['var_function'] = var_function
features_df['dom_counts'] = dom_method 

In [ ]:
features_df.head()

In [ ]:
features_df.shape

In [ ]:
#Save to file
features_df.to_csv("apriori_features.csv",index=False)